In [77]:
# Import Libraries
import numpy as np

In [78]:
# Constants
TOLERANCE = 10**-8
A, B = 1, 2.5
EXACT = -10 * (np.cos(10) - np.cos(10/2.5))

In [79]:
# Composite Simpson's Rule
def simpsons(f, a, b, n):
  h = (b - a) / n
  x_0 = f(a) + f(b)
  x_1, x_2 = 0, 0
  x = 0
  for i in range(1, n):
    x = a + i * h
    if i % 2 == 0:
      x_2 = x_2 + f(x)
    else:
      x_1 = x_1 + f(x)
  return h * (x_0 + 2 * x_2 + 4 * x_1) / 3

# Function to Integrate
def sin_function(x):
  return (100/(x**2)) * np.sin(10/x)

# Finding the least value of n using the error estimate
def simpsons_test(f, a, b):
  n = 4
  while abs(simpsons(f, a, b, n) - simpsons(f, a, b, n - 2)) > TOLERANCE:
    n += 2
  return simpsons(f, a, b, n), int(n / 2), simpsons(f, a, b, n) - simpsons(sin_function, a, b, n - 2)

# Finding the least value of n using the exact value
def simpsons_tolerance(f, a, b, tolerance):
  n = 4
  while abs(simpsons(f, a, b, n) - EXACT) > tolerance:
    n += 2
  return simpsons(f, a, b, n), int(n / 2), abs(simpsons(f, a, b, n) - EXACT) # Value, Iterations, Error

print("Composite Simpson's Rule using Error Estimate")
value, iteration, error = simpsons_test(sin_function, A, B)
print("Iterations", iteration)
print("Definite Integral:", value)
print("Error Estimate:", error)
print("Absolute Error:", abs(value - EXACT), "\n")

print("Absolute Error Between Errors", abs(error - abs(value - EXACT)), "\n")

value, iteration, error = simpsons_tolerance(sin_function, A, B, TOLERANCE)
print("Composite Simpson's Rule using Absolute Error")
print("Iterations:", iteration)
print("Definite Integral:", value)
print("Absolute Error:", error, "\n")

Composite Simpson's Rule using Error Estimate
Iterations 154
Definite Integral: 1.8542794496163608
Error Estimate: -9.701481396007239e-09
Absolute Error: 3.674879558435151e-07 

Absolute Error Between Errors 3.7718943723952236e-07 

Composite Simpson's Rule using Absolute Error
Iterations: 380
Definite Integral: 1.8542790920422032
Absolute Error: 9.913798226790504e-09 



In [80]:
# Adaptive Quadrature Method
def adaptive_quadrature(f, a, b, tolerance):
  global iterations
  global difference
  h = (b - a) / 2
  s1 = simpsons(f, a, b, 2) # ---a---------c---------b--- #
  s2 = simpsons(f, a, b, 4) # ---a----x----c----x----b--- #
  difference = s2 - s1
  if abs(difference) >= 10 * tolerance:
    iterations += 1
    c = a + h
    return adaptive_quadrature(f, a, c, tolerance) + adaptive_quadrature(f, c, b, tolerance)
  else:
    return s2 + difference/15 # Add the estimated error to make it more accurate

iterations = 0
difference = 0
quad_value = adaptive_quadrature(sin_function, A, B, TOLERANCE)
print("Adaptive Quadrature")
print("Iterations:", iterations)
print("Definite Integral:", quad_value)
print("Error Estimate:", difference/15)
print("Absolute Error:", abs(quad_value - EXACT))

Adaptive Quadrature
Iterations: 79
Definite Integral: 1.8542790819930453
Error Estimate: -3.074354200845638e-09
Absolute Error: 1.3535972342992864e-10


Before we start analyzing the results of Composite Simpson's rule and Adaptive Quadrature,
lets discuss the methods used in obtaining the results.

In the Simpson Rule we are basically integrating the lagrange polynomial of points a, b, c,
where c is the midpoint between a and b. Then we  use the taylor series to estimate the
integral since it is a lagrange polynomial. Therefore, we can use the last term, or the fourth
derivative to estimate the error of simpson's rule.

Composite Simpson's Rule applies Simpson's Rule by utilizing multiple sub intervals.
Composite Simpson's Rule makes it so that the error bound will simply be the sum all the errors
from each Simpson's Rule for the sub intervals. Furthermore, we compare the Composite
Simpson's Rule for the n sub intervals and n + 2 sub intervals.
If the difference is within the selected error bound, we deem the estimated integral as accurate.

In this assigment we must use an error bound of $10^{-8}$ or in other words, the tolerance
of the assignment is any number below $10^{-8}$. We are using this basis to estimate this
function from range 1 to 2.5.

Composite Simpson's Rule used 154 iterations for the estimate integral to be within the tolerance of
$10^{-8}$ The number of iterations seems correct because the error estimate must be within the error
bound, basically the only number the tolerance accepts is less than $10^{-8}$.
Furthermore, it is to be noted that the actual absolute error is not within the error bound
because the absolute error is 3.674879558435151e-07 which is a number clearly above the tolerance
of $10^{-8}$. Therefore, since the absolute error is not within the tolerance limits, we can
say that the error estimate is around a whole digits place less accurate than we expected
because we get an absolute error is $10^{-7}$ compared to the error bound of $10^{-8}$.

Composite Simpson's Rule using Error Estimate  
Number of Iterations: 154  
Definite Integral: 1.8542794496163608  
Error Estimate: -9.701481396007239e-09  
Absolute Error: 3.674879558435151e-07  

Since our absolute error was not within error bounds, this time let us try to
iterate until we reach a result where the absolute value also falls within the
error bound of $10^{-8}$.

Composite Simpson's Rule using Absolute Error  
Iterations: 380  
Definite Integral: 1.8542790920422032  
Absolute Error: 9.913798226790504e-09  

It took 380 iterations in order to get a result where the absolute error
was below the error bound of $10^{-8}$. Therefore, we must compare the absolute error
because otherwise our results will not satisfy the requirement of error bound, $10^{-8}$.
Furthermore, it may be of importance to take note that the error estimate of the result
with 154 iterations is similar to the absolute error of the trial with 380 iterations.

Error Estimate: -9.701481396007239e-09  
Absolute Error: 9.913798226790504e-09  

We come to the conclusion that the error estimate is similar to the actual absolute error
with the caveat being that the methods return a drastically different number of iterations.

Adaptive Quadrature and the Composite Simpson's Rule are quite similar, except the fact that
the Adaptive Quadrature only subdivides into sub intervals when it deems necessary. Utilizing the
same error estimate we subdivide depending on if the result is within error bound.

For my implementation of Adaptive Quadrature I utilized a recursive implementation over the one
provided in the textbook. Basically in my implementation the idea is to calculate S(a, b)
and S(a,c) + S(c, b) and compare the errors and recursively subdivide until the results
are within the error bound. As a result of this processess, I believe that this implementation
is more intuitive in order to find the result with lower error. In order to accomplish this
I utilized the Composite Simpson's Rule from the previous trial and in the Adaptive Quadrature
and used the output from that method accordingly.

The Results of Adaptive Quadrature:  
Iterations: 79  
Definite Integral: 1.8542790819930453  
Error Estimate: -3.074354200845638e-09  
Absolute Error: 1.3535972342992864e-10  

Clearly, Adaptive Quadrature is much better than Composite Simpson's Rule and Simpson's rule
because there are less iterations than any trial results of Composite Simpson's rule.
Furthermore, the iterations of Adaptive Quadrature are in double digits (79) while the results
of Composite Simpson's Rule are in triple digits (154 and 380). In fact Quadrature Adaptive
has an absolute error of 1.3535972342992864e-10, while Composite Simpson's rule has an absolute
error of 9.913798226790504e-09, clearly Quadrature Adaptive has a much lower error.
In conclusion, Adaptive Quadrature is much better than Compostive Simpson's rule because
there are less iterations and smaller errors.